In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 공통설정

In [1]:
import os
import re
import json
import glob
import gc
import traceback

import pandas as pd
from typing import List, Dict, Any
from datasets import Dataset

from dotenv import load_dotenv
load_dotenv()

from ragas import evaluate

# ---- AnswerRelevancy / Faithfulness import (ragas 버전 차이 대응) ----
try:
    from ragas.metrics._answer_relevancy import AnswerRelevancy
except Exception:
    from ragas.metrics import answer_relevancy as AnswerRelevancy

try:
    from ragas.metrics._faithfulness import Faithfulness
except Exception:
    from ragas.metrics import faithfulness as Faithfulness

from langchain_upstage import ChatUpstage, UpstageEmbeddings
from chatbot_app.modules.rag_module import create_pipeline


from ragas.run_config import RunConfig
rc = RunConfig(max_workers=1, timeout=180)


# -----------------------------
# LLM / Embeddings (평가용)
# -----------------------------
# ragas가 LangChain LLM wrapper를 요구하는 버전이 있어서 안전하게 래핑 시도
try:
    from ragas.llms import LangchainLLMWrapper
    ragas_llm = LangchainLLMWrapper(ChatUpstage(model="solar-pro2", temperature=0))
except Exception:
    ragas_llm = ChatUpstage(model="solar-pro2", temperature=0)

ragas_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
upstage_embedding = UpstageEmbeddings(model="solar-embedding-1-large")

print(f"ragas_llm={type(ragas_llm)}")
print(f"ragas_embeddings={type(ragas_embeddings)}")

# -----------------------------
# Metrics
# -----------------------------
def _metric_instance(m):
    # m이 클래스면 m()로 인스턴스 만들고,
    # 이미 인스턴스/객체면 그대로 반환
    return m() if callable(m) else m


AR_METRIC = _metric_instance(AnswerRelevancy)
F_METRIC = _metric_instance(Faithfulness)

OUT_DIR = "./data/RAGAS"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# 몽키패치: AnswerRelevancy 커널 크래시(세그폴트) 회피용
# - AnswerRelevancy 내부 cosine similarity 계산이 numpy/BLAS 경로를 타면
#   Windows 환경에서 드물게 하드 크래시가 발생할 수 있어,
#   similarity 계산만 "완전 파이썬" 구현으로 우회합니다.
# - 또한 providers 제약(Upstage: n must be 1) 때문에 strictness=1을 강제합니다.
# -----------------------------
import math
import numpy as np

import ragas.metrics._answer_relevance as ar

def _cos_sim(u, v, eps=1e-8):
    # numpy dot/BLAS 안 쓰는 순수 파이썬 코사인
    su = 0.0
    sv = 0.0
    s  = 0.0
    for a, b in zip(u, v):
        af = float(a); bf = float(b)
        s  += af * bf
        su += af * af
        sv += bf * bf
    return s / (math.sqrt(su) * math.sqrt(sv) + eps)

def safe_calculate_similarity(self, question: str, generated_questions: list[str]):
    """
    ragas AnswerRelevancy/ResponseRelevancy가 기대하는 시그니처:
      (self, question: str, generated_questions: list[str]) -> array-like with .mean()
    """
    # 1) metric이 들고 있는 embeddings 잡기
    emb = None
    for name in ["embeddings", "_embeddings", "embedding", "_embedding"]:
        if hasattr(self, name):
            emb = getattr(self, name)
            if emb is not None:
                break

    if emb is None:
        raise RuntimeError("AnswerRelevancy metric에 embeddings가 세팅되지 않았습니다.")

    # 2) 임베딩 함수 선택 (sync만)
    def embed_one(text: str):
        if hasattr(emb, "embed_query"):
            return emb.embed_query(text)
        if hasattr(emb, "embed_documents"):
            return emb.embed_documents([text])[0]
        raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다(동기 메서드 필요).")

    def embed_many(texts: list[str]):
        if len(texts) == 0:
            return []
        if hasattr(emb, "embed_documents"):
            return emb.embed_documents(texts)
        # embed_query만 있으면 루프
        if hasattr(emb, "embed_query"):
            return [emb.embed_query(t) for t in texts]
        raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다(동기 메서드 필요).")

    # 3) 문자열 -> 벡터
    qv = embed_one(question)
    gvs = embed_many(generated_questions)

    if not gvs:
        return np.array([0.0], dtype="float32")

    # 4) 코사인 유사도 (BLAS 안 탐)
    sims = [_cos_sim(qv, gv) for gv in gvs]

    # ragas는 뒤에서 cosine_sim.mean()을 호출하므로 numpy array로 반환
    return np.array(sims, dtype="float32")

# ✅ 클래스 메서드를 덮어써야 함 (모듈 함수 패치가 아님)
ar.ResponseRelevancy.calculate_similarity = safe_calculate_similarity
ar.AnswerRelevancy.calculate_similarity = safe_calculate_similarity


# 1) 모듈 레벨 함수가 있다면 __code__ 교체
if ar is not None and hasattr(ar, "calculate_similarity"):
    ar.calculate_similarity.__code__ = _cos_sim.__code__
    ar.calculate_similarity.__defaults__ = _cos_sim.__defaults__
    print("✅ patched ar.calculate_similarity (__code__)")
elif ar is not None:
    # 2) 모듈에 없으면 이름 주입(lookup 경로에 따라 먹을 수 있음)
    ar.__dict__["calculate_similarity"] = _cos_sim
    print("✅ injected ar.calculate_similarity")

    # 3) 클래스 메서드로 계산하는 버전 대비: ResponseRelevancy.calculate_similarity를 덮어쓰기
    if hasattr(ar, "ResponseRelevancy"):
        try:
            ar.ResponseRelevancy.calculate_similarity = _cos_sim
            print("✅ patched ResponseRelevancy.calculate_similarity")
        except Exception:
            pass

# Upstage 제약: n=1 강제
if hasattr(AR_METRIC, "strictness"):
    AR_METRIC.strictness = 1
    print("✅ AR_METRIC.strictness fixed to 1")

# -----------------------------
# 공통 유틸
# -----------------------------
def clip_rows(rows, *, max_ctx=4, max_chars=1200, clip_answer_chars=700):
    """요구사항 반영: context clip 1200 chars"""
    clipped = []
    for r in rows:
        rr = dict(r)
        ctx = rr.get("contexts", []) or []
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]
        if rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]
        clipped.append(rr)
    return clipped


def cleanup_memory():
    gc.collect()
    print("🧹 Memory cleaned")


def save_df(df: pd.DataFrame, out_path: str):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ saved: {out_path}")
    return out_path


# -----------------------------
# 정성 체크(평가표 컬럼): 근거표현/문장구조/추론확장/불확실성
# - 엄밀한 metric이 아니라 "패턴 탐지용" 보조 컬럼
# -----------------------------
_KR_EVIDENCE_PHRASES = ["따르면", "근거", "참고", "에 따르면", "조항", "규정", "판례", "법원", "사건번호", "제"]
_KR_UNCERTAINTY_PHRASES = ["확인 필요", "추가 확인", "가능", "일 수", "자료 범위", "제공된 자료", "단정", "불명확"]
_KR_INFERENCE_PHRASES = ["일반적으로", "통상", "보통", "대개", "추정", "원칙적으로", "추론", "상식적으로"]


def _has_any(text: str, phrases: List[str]) -> int:
    t = text or ""
    return int(any(p in t for p in phrases))


def _first_sentence(text: str) -> str:
    if not text:
        return ""
    for sep in ["\n", ".", "?", "!"]:
        if sep in text:
            return text.split(sep, 1)[0].strip()
    return text.strip()


def _looks_conclusion_first(answer: str) -> int:
    s1 = _first_sentence(answer)
    if not s1:
        return 0
    return int(any(x in s1 for x in ["입니다", "됩니다", "가능합니다", "불가능합니다", "하세요", "권합니다", "위험", "유리", "불리"]))


def _hallucinated_citation_flag(answer: str, contexts: List[str]) -> int:
    """
    답변에 'OOO 제NN조' 같은 패턴이 있는데
    컨텍스트 전체 문자열에 동일한 구절이 하나도 없으면 1 (의심)
    """
    if not answer:
        return 0
    ctx_all = "\n".join(contexts or [])
    pats = re.findall(r"([가-힣A-Za-z0-9·\s]{2,40})\s(제?\d+조(?:의\d+)?)", answer)
    if not pats:
        return 0
    for left, art in pats:
        needle = (left.strip() + " " + art.strip()).strip()
        if needle and needle not in ctx_all:
            return 1
    return 0


# -----------------------------
# 정규화 결과 “1줄 강제” 유틸
# (주의: 지금 패치는 trace 저장값만 1줄로 만들고, pipeline 내부 retrieval query에는 영향을 주지 않습니다.
#        pipeline 내부에 적용하려면 rag_module.py 쿼리 생성 직전에 동일 로직을 넣어야 합니다.)
# -----------------------------
def sanitize_normalized_query(text: str, max_chars: int = 400) -> str:
    if not text:
        return ""
    first = text.strip().splitlines()[0].strip()
    return first[:max_chars]


# ✅ 데이터 검증 함수 추가
def validate_data(rows, required_fields):
    """데이터 검증 및 문제 리포트"""
    issues = []
    for i, r in enumerate(rows):
        for field in required_fields:
            if field not in r:
                issues.append(f"Row {i}: missing field '{field}'")
            elif r[field] is None:
                issues.append(f"Row {i}: field '{field}' is None")
            elif field in ["question", "answer"] and not str(r[field]).strip():
                issues.append(f"Row {i}: field '{field}' is empty")
    
    if issues:
        print("⚠️ Data validation issues:")
        for issue in issues[:10]:  # 처음 10개만 출력
            print(f"  - {issue}")
        if len(issues) > 10:
            print(f"  ... and {len(issues) - 10} more issues")
    else:
        print("✅ Data validation passed")
    
    return len(issues) == 0

# -----------------------------
# 실행 태그 / 디렉토리
# -----------------------------
def make_run_tag_for_answer(model_name: str) -> str:
    return f"llmcmp_{model_name}"


RUNS_ROOT = os.path.join(OUT_DIR, "runs_answer")
os.makedirs(RUNS_ROOT, exist_ok=True)


# -----------------------------
# run_once_answer: AnswerRelevancy / Faithfulness 1회
# -----------------------------
def run_once_answer(
    pipeline,
    items: List[Dict[str, Any]],
    run_idx: int,
    llm_name: str = "solar-pro2",
) -> Dict[str, Any]:
    run_tag = make_run_tag_for_answer(llm_name)
    run_dir = os.path.join(RUNS_ROOT, run_tag, f"run_{run_idx:02d}")
    os.makedirs(run_dir, exist_ok=True)

    cache_jsonl = os.path.join(run_dir, "rag_run_cache.jsonl")
    print(f"--- run_once_answer START | run_idx={run_idx} | llm_name={llm_name} ---")
    # (1) 파이프라인 실행 결과 캐시 저장
    with open(cache_jsonl, "w", encoding="utf-8") as f:
        print(f"items={len(items)}")
        for i, ex in enumerate(items, start=1):
            q = ex["question"]
            gt = ex.get("ground_truth", "")

            MAX_QUERY_CHARS = 2000
            q_for_retrieval = q if len(q) <= MAX_QUERY_CHARS else q[:MAX_QUERY_CHARS]

            try:
                trace = pipeline.answer_with_trace(q_for_retrieval, skip_normalization=False)
                if "normalized_query" in trace:
                    trace["normalized_query"] = sanitize_normalized_query(trace.get("normalized_query", ""))
            except Exception as e:
                trace = {"answer": "", "docs": [], "normalized_query": ""}

            answer = trace.get("answer", "") or ""
            docs = trace.get("docs", []) or []

            contexts = []
            for d in docs:
                try:
                    contexts.append(d.page_content)
                except Exception:
                    pass

            row = {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "reference": gt,
                "ground_truths": [gt],
                "normalized_query": trace.get("normalized_query", ""),
            }
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    # (2) rows 로드 + 최소 정리
    raw_rows = []
    with open(cache_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                raw_rows.append(json.loads(line))

    rows_local = []
    for r in raw_rows:
        rr = dict(r)
        if not rr.get("question"):
            continue
        if not rr.get("answer"):
            rr["answer"] = "답변을 생성할 수 없습니다."
        if "contexts" not in rr or rr["contexts"] is None:
            rr["contexts"] = []
        if not isinstance(rr["contexts"], list):
            rr["contexts"] = [str(rr["contexts"])]
        rows_local.append(rr)

    validate_data(rows_local, required_fields=["question", "answer", "contexts"])

    # (3) clip (요구사항: max_chars=1200)
    rows_eval = clip_rows(rows_local, max_ctx=4, max_chars=1200, clip_answer_chars=700)

    ds = Dataset.from_list(
        [
            {
                "question": r["question"],
                "answer": r["answer"],
                "contexts": r.get("contexts", []),
                "reference": r.get("reference", ""),
            }
            for r in rows_eval
        ]
    )

    # (4) RAGAS 평가
    print(f"evaluate START | run_idx={run_idx}")
    import sys, ragas.metrics._answer_relevance as ar

    print("ar module id:", id(ar))
    print("calc filename:", ar.calculate_similarity.__code__.co_filename)
    print("in sys.modules:", sys.modules.get("ragas.metrics._answer_relevance") is ar)

    AR_METRIC.strictness = 1
    res = evaluate(
        dataset=ds,
        metrics=[AR_METRIC, F_METRIC],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        run_config=rc,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )


    print("evaluate RETURNED")   # ✅ 이 줄이 찍히는지 여부가 핵심
    df = res.to_pandas()

    print("to_pandas DONE")      # ✅ 이 줄이 찍히는지 여부도 핵심
    # id 컬럼 보정
    if "id" not in df.columns:
        df = df.copy()
        df["id"] = range(1, len(df) + 1)

    # (5) 정성 체크 컬럼 추가
    id_to_ctx = {i + 1: rows_eval[i].get("contexts", []) for i in range(len(rows_eval))}
    id_to_ans = {i + 1: rows_eval[i].get("answer", "") for i in range(len(rows_eval))}

    df["evidence_expression"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_EVIDENCE_PHRASES))
    df["structure_conclusion_first"] = df["id"].map(lambda _id: _looks_conclusion_first(id_to_ans.get(_id, "")))
    df["inference_expansion_signal"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_INFERENCE_PHRASES))
    df["uncertainty_signal"] = df["id"].map(lambda _id: _has_any(id_to_ans.get(_id, ""), _KR_UNCERTAINTY_PHRASES))
    df["hallucinated_citation_suspect"] = df["id"].map(
        lambda _id: _hallucinated_citation_flag(id_to_ans.get(_id, ""), id_to_ctx.get(_id, []))
    )

    # (저장1) ragas raw
    out_raw = os.path.join(run_dir, "ragas_result_answer_faith.csv")
    save_df(df, out_raw)

    # (저장2) per_question (보기 좋은 평가표)
    perq_cols = [
        "id",
        "answer_relevancy",
        "faithfulness",
        "evidence_expression",
        "structure_conclusion_first",
        "inference_expansion_signal",
        "uncertainty_signal",
        "hallucinated_citation_suspect",
    ]
    # ✅ 실제 df에 존재하는 컬럼만 선택 (metrics를 바꿔도 KeyError 방지)
    _cols_exist = [c for c in perq_cols if c in df.columns]
    perq = df[_cols_exist].copy()
    for c in ["answer_relevancy", "faithfulness"]:
        if c in perq.columns:
            perq[c] = perq[c].astype(float).round(3)

    perq_csv = os.path.join(run_dir, "per_question.csv")
    save_df(perq, perq_csv)

    print(f"saved per_question_csv={perq_csv}")
    print(f"--- run_once_answer DONE | run_idx={run_idx} ---")
    cleanup_memory()

    return {
        "run_tag": run_tag,
        "run_idx": run_idx,
        "run_dir": run_dir,
        "cache_jsonl": cache_jsonl,
        "raw_csv": out_raw,
        "per_question_csv": perq_csv,
        "AR_mean": float(perq["answer_relevancy"].mean()),
        "AR_std": float(perq["answer_relevancy"].std(ddof=1)) if len(perq) > 1 else 0.0,
        "F_mean": float(perq["faithfulness"].mean()),
        "F_std": float(perq["faithfulness"].std(ddof=1)) if len(perq) > 1 else 0.0,
    }


# -----------------------------
# 반복 실행(run_repeat_answer) + wide CSV
# -----------------------------
def run_repeat_answer(pipeline, items: List[Dict[str, Any]], n: int = 3, llm_name: str = "solar-pro2") -> List[Dict[str, Any]]:
    results = []
    for run_idx in range(1, n + 1):
        print("\n==============================")
        print(f"RUN {run_idx:02d}/{n} | {make_run_tag_for_answer(llm_name)}")
        print("==============================")
        results.append(run_once_answer(pipeline, items, run_idx, llm_name=llm_name))
    return results


def build_wide_csv_answer(run_tag: str, results: List[Dict[str, Any]]) -> str:
    """
    메모리 안정성을 위해 결과 dict에 df_perq를 저장하지 않고,
    per_question_csv를 다시 읽어서 wide를 만듭니다.
    """
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    os.makedirs(base_dir, exist_ok=True)

    ids = pd.DataFrame({"id": range(1, 11)})
    wide = ids.copy()

    for res in results:
        ridx = res["run_idx"]
        dfm = pd.read_csv(res["per_question_csv"])[[
            "id",
            "answer_relevancy",
            "faithfulness",
            "hallucinated_citation_suspect",
        ]].copy()

        dfm = dfm.rename(columns={
            "answer_relevancy": f"r{ridx}-answer_relevancy",
            "faithfulness": f"r{ridx}-faithfulness",
            "hallucinated_citation_suspect": f"r{ridx}-hallucination_suspect",
        })
        wide = pd.merge(wide, dfm, on="id", how="left")

    wide = wide.set_index("id").sort_index()

    ar_cols = [c for c in wide.columns if c.endswith("-answer_relevancy")]
    f_cols = [c for c in wide.columns if c.endswith("-faithfulness")]

    wide["answer_relevancy-mean"] = wide[ar_cols].mean(axis=1, skipna=True)
    wide["faithfulness-mean"] = wide[f_cols].mean(axis=1, skipna=True)

    mean_row = wide.mean(axis=0, skipna=True).to_frame().T
    mean_row.index = ["mean"]
    wide2 = pd.concat([wide, mean_row], axis=0)

    out_csv = os.path.join(base_dir, "runs_wide_3repeats.csv")
    wide2.to_csv(out_csv, encoding="utf-8")
    print("✅ saved:", out_csv)
    return out_csv


# -----------------------------
# DOCX 질문 로딩 (업로드 파일 경로 기준)
# -----------------------------
from docx import Document

DOCX_PATH = "data/RAGAS/RAGAS_질문_25개_정리본.docx"

def load_items_from_docx(docx_path: str, max_q: int = 25):
    doc = Document(docx_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]

    items = []
    i = 0
    while i < len(paras) and len(items) < max_q:
        line = paras[i]
        m = re.match(r"^(\d+)\.\s*(.+)$", line)
        if not m:
            i += 1
            continue

        question = m.group(2).strip()

        i += 1
        answer_lines = []
        while i < len(paras):
            if re.match(r"^\d+\.\s*.+$", paras[i]):
                break
            answer_lines.append(paras[i])
            i += 1

        gt = "\n".join([x for x in answer_lines if x not in ["✔️ 모범답안"]]).strip()

        items.append({"question": question, "ground_truth": gt})

    return items


items = load_items_from_docx(DOCX_PATH, max_q=25)
print("✅ items loaded:", len(items))
print("sample Q1:", items[0]["question"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_15524\906357061.py:21: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import answer_relevancy as AnswerRelevancy
C:\Users\Admin\AppData\Local\Temp\ipykernel_15524\906357061.py:42: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatUpstage(model="solar-pro2", temperature=0))


ragas_llm=<class 'ragas.llms.base.LangchainLLMWrapper'>
ragas_embeddings=<class 'langchain_upstage.embeddings.UpstageEmbeddings'>
✅ injected ar.calculate_similarity
✅ patched ResponseRelevancy.calculate_similarity
✅ AR_METRIC.strictness fixed to 1
✅ items loaded: 25
sample Q1: 전입신고·확정일자 했는데, 확정일자부 내용까지 중요한가요?


# 2. 실행

In [2]:
%%time

from langchain_openai import ChatOpenAI

gpt_llm = ChatOpenAI(
    model="gpt-4o-mini",
    # model="gpt-5-mini",
    temperature=0
)

upstage_llm = ChatUpstage(
    model="solar-pro2",
    temperature=0
)

pipeline = create_pipeline(
    generation_llm=gpt_llm,
    # generation_llm=upstage_llm,
    embedding=upstage_embedding,   
)

# -----------------------------
# 1회 실행 (반복 실험 제거)
# -----------------------------
result = run_once_answer(pipeline, items, run_idx=1, llm_name="gpt-4o-mini")

# 저장 파일들 경로 출력
print("✅ DONE. run_tag =", result.get("run_tag"))
print(" - raw_csv:", result.get("raw_csv"))
print(" - perq_csv:", result.get("perq_csv"))
print(" - summary_csv:", result.get("summary_csv"))


2026-02-09 16:12:54,767 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...
2026-02-09 16:12:56,472 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-02-09 16:12:56,497 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)


--- run_once_answer START | run_idx=1 | llm_name=gpt-4o-mini ---
items=25


2026-02-09 16:12:57,582 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:12:57,594 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자 했는데, 확정일자부 내용까지 중요한가요? (주민등록·확정일자 했는데, 확정일자부 내용까지 중요한가요?)  

※ [용어 사전]에 '전입신고'는 '주민등록'으로만 매핑되어 있으며, '확정일자부'는 별도 매핑 대상이 아니므로 원문을 유지하였습니다.
2026-02-09 16:12:57,595 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자 했는데, 확정일자부 내용까지 중요한가요? (주민등록·확정일자 했는데, 확정일자부 내용까지 중요한가요?)  

※ [용어 사전]에 '전입신고'는 '주민등록'으로만 매핑되어 있으며, '확정일자부'는 별도 매핑 대상이 아니므로 원문을 유지하였습니다.'
2026-02-09 16:12:58,121 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:00,577 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:02,962 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:05,227 - httpx - INFO - HTT

2026-02-09 16:13:26,873 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:27,593 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:28,482 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:28,717 - chatbot_app.modules.rag_module - INFO - 📝 Using prompt mode: GENERAL
2026-02-09 16:13:28,717 - chatbot_app.modules.rag_module - INFO - 🤖 답변 생성 중...
2026-02-09 16:13:32,211 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:13:33,400 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:13:33,404 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 묵시적갱신(묵시적갱신)으로 연장된 줄 모르고 살았는데, 주택의인도(주택의인도) 가려면 언제까지 살아야 하나요?  

※ 참고: '이사'의 경우 문맥상 '주택의인도'로 변환되었으나, 정확한 법률 용어 적용을 위해 추가 확인이 필요할 수 있습니다. (

2026-02-09 16:13:57,153 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:57,905 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:58,658 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:13:59,192 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 429 Too Many Requests"
2026-02-09 16:13:59,192 - chatbot_app.modules.rag_module - WARNING - ⚠️ Rerank 실패 (skip): headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '51471c75d52742fa3ce1f2267aca7f00', 'x-trial-endpoint-call-l

...

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Batch 1/50:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-09 16:17:36,005 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:17:37,282 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:17:38,093 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:17:40,062 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:17:48,079 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:17:49,467 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:17:50,715 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:17:51,603 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-

2026-02-09 16:20:31,346 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:20:32,277 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:20:34,744 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:20:41,663 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:20:43,344 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-09 16:20:44,625 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:20:45,719 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-09 16:20:47,828 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-

evaluate RETURNED
to_pandas DONE
✅ saved: ./data/RAGAS\runs_answer\llmcmp_gpt-4o-mini\run_01\ragas_result_answer_faith.csv
✅ saved: ./data/RAGAS\runs_answer\llmcmp_gpt-4o-mini\run_01\per_question.csv
saved per_question_csv=./data/RAGAS\runs_answer\llmcmp_gpt-4o-mini\run_01\per_question.csv
--- run_once_answer DONE | run_idx=1 ---
🧹 Memory cleaned
✅ DONE. run_tag = llmcmp_gpt-4o-mini
 - raw_csv: ./data/RAGAS\runs_answer\llmcmp_gpt-4o-mini\run_01\ragas_result_answer_faith.csv
 - perq_csv: None
 - summary_csv: None
CPU times: total: 15.5 s
Wall time: 10min 6s


In [3]:
df = pd.read_csv('data/RAGAS/runs_answer/llmcmp_gpt-4o-mini/run_01/per_question.csv')
df

,id,answer_relevancy,faithfulness,evidence_expression,structure_conclusion_first,inference_expansion_signal,uncertainty_signal,hallucinated_citation_suspect
0,1,0.417,0.222,1,0,0,1,1
1,2,0.396,0.071,1,0,0,1,1
2,3,0.502,0.125,1,0,0,1,1
3,4,0.550,0.214,1,0,0,1,1
4,5,0.537,0.273,1,0,0,1,1
5,6,0.320,0.000,1,0,0,1,1
6,7,0.500,0.300,1,0,0,1,1
7,8,0.489,0.125,1,0,0,1,1
8,9,0.175,0.182,1,0,0,1,1
9,10,0.255,0.182,1,0,0,1,1


In [5]:
# df = pd.read_csv('data/RAGAS/runs_answer/llmcmp_solar-pro2/run_01/per_question.csv')
# df

,id,answer_relevancy,faithfulness,evidence_expression,structure_conclusion_first,inference_expansion_signal,uncertainty_signal,hallucinated_citation_suspect
0,1,0.000,0.600,1,0,0,1,1
1,2,0.407,0.400,1,0,1,1,1
2,3,0.000,0.714,1,0,0,0,1
3,4,0.492,0.875,1,1,0,1,1
4,5,0.625,0.286,1,0,0,0,1
5,6,0.326,0.600,1,1,0,1,1
6,7,0.483,0.500,1,0,0,0,1
7,8,0.604,0.286,1,0,0,1,1
8,9,0.000,0.167,1,0,0,0,1
9,10,0.000,0.556,1,0,0,1,1
